In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py
import numpy as np
import numpy.linalg as nl
# 기호 연산 기능 추가
# Add symbolic operation capability
import sympy as sy

In [ ]:
sy.init_printing()

# 상미분방정식을 위한 수정 오일러법 (훈의 방법)<br>Modified Euler Method (Heun's Method) for Ordinary Differntial Equations

## 전진오일러법 사례 검토<br>Review of Forward Euler Method

다시 한번 다음 1계 미분 방정식을 생각해 보자.<br>Once again, let's think about the following first order differential equation again.

$$
\left\{
    \begin{align}
        a_0 \frac{d}{dt}x(t)+a_1 x(t)&=0 \\
        x(0)&=x_0 \\
    \end{align}
\right.
$$

python 함수로는 다음과 같이 쓸 수 있다.<br>We can write a function in python as follows.

In [ ]:
a_0, a_1 = 2.0, 1.0

def dx_dt(x, t):
    return - a_1 * x / a_0

$0 \le t \le 6$ 구간에서 $\Delta t=1(sec)$ 으로 전진 오일러법을 적용해 보자.

In [ ]:
def forward_euler(f, t_array, x_0):
    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        delta_t = t_array[k+1] - t_array[k]

        # slope
        s_i = f(x_i, t_i)

        # x[i + 1]
        x_i_plus_1 = x_i + s_i * delta_t

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1)
        
        x_i = x_i_plus_1

    return time_list, result_list


In [ ]:
# Time step interval
delta_t = 1.0

# Time array
t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)

# Initial state
x_0 = 4.5

# *** ODE solver ***
t_out, x_out = forward_euler(dx_dt, t_sec_array, x_0)

py.plot(t_out, x_out, '.-', label='Forward Euler')

py.axis('equal')

py.xlabel('t(sec)')
py.ylabel('x(m)')

py.legend(loc=0)
py.grid(True)


$0 \le t \le 6$, $0 \le x \le 6$ 인 영역에서 기울기와 겹쳐 그려 보자.<br>
Let's plot overlap the slopes within the region of $0 \le t \le 6$ and $0 \le x \le 6$.

In [ ]:
t_array = py.linspace(0, 6)
x_array = py.linspace(0, 6)

In [ ]:
def ode_slope_1state_interval(func, delta_t, delta_x, te, ti, x_max, x_min):
    time_list = np.arange(ti, te, delta_t)
    x_list = np.arange(x_min, x_max + 0.5 * delta_x, delta_x)
    ode_slope_1state(func, x_list, time_list)
    return time_list


def ode_slope_1state(func, x_list, time_list):
    """
    Plot field of arrows indicating derivatives of the state
    :param func:
    :param x_list:
    :param time_list:
    :return:
    """
    time_mesh, x_mesh = np.meshgrid(time_list, x_list)
    u_mesh = np.ones_like(x_mesh)
    v_mesh = func(x_mesh, time_mesh)
    # magnitude as color
    color_mesh = np.sqrt(u_mesh * u_mesh + v_mesh * v_mesh)

    # https://stackoverflow.com/questions/29589119/plot-width-settings-in-ipython-notebook
    py.figure(figsize=(12, 12))
    py.quiver(time_mesh, x_mesh, u_mesh, v_mesh, color_mesh, angles='xy')
    py.xlabel('t')
    py.ylabel('x')
    py.xlim((time_list[0] - (time_list[1] - time_list[0]) * 0.125, time_list[-1]))
    py.ylim((min(x_list) - (x_list[1] - x_list[0]) * 0.125,
                max(x_list) + (x_list[-1] - x_list[-2]) * 0.125))
    py.grid(True)

In [ ]:
# Slopes at each (t, x) points
ode_slope_1state(dx_dt, x_array, t_array)

# Time step interval
delta_t = 1.0

# Time array
t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)

# Initial state
x_0 = 4.5

# *** ODE solver ***
t_out, x_out = forward_euler(dx_dt, t_sec_array, x_0)

py.plot(t_out, x_out, '.-', label='Forward Euler')

py.axis('equal')

py.xlabel('t(sec)')
py.ylabel('x(m)')

py.legend(loc=0)
py.grid(True)


$\Delta t=1(sec)$ 의 경우, $t_0=0(sec)$ 과 $t_1=1(sec)$ 사이에서 $\frac{d}{dt}x$는 계속 변화하고 있으나, 전진 오일러법은 $t_0 \le t \le t_1$ 사이에서 $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_0}$로 가정한다.<br>
In case of $\Delta t=1(sec)$, $\frac{d}{dt}x$ continuously changes between $t_0=0(sec)$ and $t_1=1(sec)$.  However, the Forward Euler Method assumes that $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_0}$ over $t_0 \le t \le t_1$.

그렇다면, $t_i \le t \le t_{i+1}$ 사이에서 대표적인 $\frac{d}{dt}x$ 값은 어떤 값이 좋을 것인가?<br>
If so, within $t_i \le t \le t_{i+1}$, which value of $\frac{d}{dt}x$ would be representative?

## 수정 오일러법 (훈의 방법)<br>Modified Euler Method (Heun's Method)

수정 오일러법은 $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_{i}}$ 와 $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_{i+1}}$의 평균값이 $t_i \le t \le t_{i+1}$ 사이에서 대표적인 $\frac{d}{dt}x$ 값으로 가정한다.<br>
Modified Euler Method assumes that the average of $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_{i}}$ and $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_{i+1}}$ as the representative $\frac{d}{dt}x$ value within $t_i \le t \le t_{i+1}$ interval.

그런데, $x(t)$의 엄밀해를 알지 못하는 상태에서 어떻게 $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_{i+1}}$ 을 계산할 것인가?<br>
Now, how can we calculate $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_{i+1}}$ without knowing the exact solution of $x(t)$?

전진 오일러법으로 구한 $\left.x\right|_{t=t_{i+1}}$의 근사값 $\left.\hat{x}\right|_{t=t_{i+1}}$으로 $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_{i+1}}$을 사용할 것이다.<br>
We would use $\frac{d}{dt}x=\left.\frac{d}{dt}x\right|_{t=t_{i+1}}$ using $\left.\hat{x}\right|_{t=t_{i+1}}$, the approximation of $\left.x\right|_{t=t_{i+1}}$ by the Forward Euler Method.

In [ ]:
def modified_euler(f, t_array, x_0):
    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        delta_t = t_array[k+1] - t_array[k]

        # slope at i
        s_i = f(x_i, t_i)

        # x[i + 1] by Forward Euler
        x_i_plus_1 = x_i + s_i * delta_t
        
        # slope at i + 1
        s_i_plus_1 = f(x_i_plus_1, t_array[k+1])
        
        # average of slope
        s_average = (s_i + s_i_plus_1) * 0.5
        
        # x[i + 1] by Modified Euler
        x_i_plus_1_m = x_i + s_average * delta_t

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1_m)
        
        x_i = x_i_plus_1_m

    return time_list, result_list


엄밀해, 전진 오일러법, 수정 오일러법을 비교해 보자.<br>
Let's compare the exact solution, Forware Euler Method, and Modified Euler Method.

In [ ]:
def exact(t):
    return x_0 * py.exp((-a_1 / a_0) * t)


In [ ]:
# Initial value
x_0 = 4.5

# Slopes
ode_slope_1state(dx_dt, x_array, t_array)

# Exact solution
x_exact_array = exact(t_array)
py.plot(t_array, x_exact_array, label='exact')

# Forward Euler
py.plot(t_out, x_out, '.-', label='Forward Euler')

# Modified Euler
t_m_out, x_m_out = modified_euler(dx_dt, t_sec_array, x_0)
py.plot(t_m_out, x_m_out, '*-', label='Modified Euler')

py.legend(loc=0, fontsize='xx-large')


전진 오일러법에 비해 수정 오일러법의 근사해가 엄밀해에 비해 오차가 더 적은 것을 알 수 있다.<br>
We can see that the approximate solution of Modified Euler Method is closer to the exact solution than that the Forward Euler Method.

수정 오일러법은 독일 수학자 칼 훈의 이름을 따서 훈의 방법이라고도 부른다.<br>
Sometimes, Modified Euler Method is called Heun's method, named after German mathematician Karl Heun.

## 도전 과제<br>Try This

01 다음 미분방정식의 엄밀해를 구하시오:<br>
Find exact solution of the following differential equation:

$$
\begin{align}
10 \frac{d}{dt}x(t) + 100 x(t) &= 0 \\
x(0) &= 0
\end{align}
$$


위 미분방정식의 수치해를 전진 오일러법으로 구하시오.<br>
Find numerical solution of the above differential equation using Forward Euler Method.



위 미분방정식의 수치해를 훈의 방법으로 구하고 엄밀해, 전진 오일러법과 비교하시오.<br>
Find numerical solution of the above differential equation using Heun's method and compare with exact solution and Forward Euler Method.



02 다음 미분방정식의 수치해를 전진 오일러법으로 구하시오:<br>
Find numerical solution of the following differential equation using Forward Euler Method:

$$
\begin{align}
10 \frac{d}{dt}x(t) + 100 x(t) &= sin(t[rad]) \\
x(0) &= 0
\end{align}
$$


위 미분방정식의 수치해를 훈의 방법으로 구하고 전진 오일러법과 비교하시오.<br>
Find numerical solution of the above differential equation using Heun's method and compare with Forward Euler Method.

